In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
pd.set_option('display.max_columns', None)
from sklearn.feature_selection import RFE
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Inspect

In [3]:
# importing original data and previously worked dataframe 
train = pd.read_csv("../project_data/original_data/train.csv")
df_train = pd.read_csv("../project_data/handout_data/training_handout.csv")

In [3]:
df_train.columns

Index(['Unnamed: 0', 'brand', 'model', 'price', 'transmission', 'mileage',
       'fuelType', 'tax', 'mpg', 'engineSize', 'car_age', 'mileage_per_year'],
      dtype='object')

In [50]:
print(train.shape)
df_train.shape

(75973, 14)


(75973, 12)

In [4]:
# remove unnamed 0 column
df_train = df_train.drop(columns=['Unnamed: 0'])
df_train

,brand,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age,mileage_per_year
0,vw,golf,22290,semi-auto,28421.0,petrol,NaN,11.4,2.0,4.0,7105.250000
1,toyota,yaris,13790,manual,4589.0,petrol,145.0,47.9,1.5,1.0,4589.000000
2,audi,q2,24990,semi-auto,3624.0,petrol,145.0,40.9,1.5,1.0,3624.000000
3,ford,fiesta,12500,manual,9102.0,petrol,145.0,65.7,1.0,2.0,4551.000000
4,bmw,2 series,22995,manual,1000.0,petrol,145.0,42.8,1.5,1.0,1000.000000
...,...,...,...,...,...,...,...,...,...,...,...
75968,mercedes,c class,13498,manual,14480.0,petrol,125.0,53.3,2.0,5.0,2896.000000
75969,audi,q3,12495,semi-auto,52134.0,diesel,200.0,47.9,2.0,7.0,7447.714286
75970,toyota,aygo,8399,automatic,11304.0,petrol,145.0,67.0,1.0,3.0,3768.000000
75971,audi,q3,12990,manual,69072.0,diesel,125.0,60.1,2.0,5.0,13814.400000


In [5]:
# Drop mileage per year cause i dont feel its goudd
df_train = df_train.drop(columns=["mileage_per_year"])
df_train

,brand,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age
0,vw,golf,22290,semi-auto,28421.0,petrol,NaN,11.4,2.0,4.0
1,toyota,yaris,13790,manual,4589.0,petrol,145.0,47.9,1.5,1.0
2,audi,q2,24990,semi-auto,3624.0,petrol,145.0,40.9,1.5,1.0
3,ford,fiesta,12500,manual,9102.0,petrol,145.0,65.7,1.0,2.0
4,bmw,2 series,22995,manual,1000.0,petrol,145.0,42.8,1.5,1.0
...,...,...,...,...,...,...,...,...,...,...
75968,mercedes,c class,13498,manual,14480.0,petrol,125.0,53.3,2.0,5.0
75969,audi,q3,12495,semi-auto,52134.0,diesel,200.0,47.9,2.0,7.0
75970,toyota,aygo,8399,automatic,11304.0,petrol,145.0,67.0,1.0,3.0
75971,audi,q3,12990,manual,69072.0,diesel,125.0,60.1,2.0,5.0


### Missing Data

In [6]:
miss = df_train.isna().sum()
mv = pd.DataFrame({"missing": miss[miss>0], "missing_%": (miss[miss>0]/len(df_train)*100).round(2)}).sort_values("missing_%", ascending=False)
if not mv.empty:
    print("\nMissing-values summary (non-zero):")
    print(mv.head(20))



Missing-values summary (non-zero):
              missing  missing_%
tax              8282      10.90
mpg              7962      10.48
transmission     2263       2.98
car_age          1849       2.43
mileage          1832       2.41
engineSize       1780       2.34
model            1721       2.27
fuelType         1678       2.21
brand              43       0.06


In [7]:
# these lines are worth dog shait and are 0.06 % of data
df_train[df_train["brand"].isnull()]

,brand,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age
442,NaN,NaN,8600,manual,28820.0,diesel,0.0,74.4,1.5,3.0
2031,NaN,NaN,21300,automatic,11258.0,NaN,145.0,64.2,2.0,2.0
2223,NaN,NaN,12990,automatic,43000.0,diesel,145.0,54.3,2.1,7.0
4250,NaN,NaN,25495,semi-auto,4255.0,petrol,145.0,44.8,1.3,1.0
5513,NaN,NaN,8997,manual,11200.0,petrol,30.0,58.0,1.3,3.0
5967,NaN,NaN,19750,semi-auto,9500.0,petrol,145.0,55.4,1.5,2.0
6233,NaN,NaN,29250,automatic,10055.0,diesel,145.0,47.9,2.0,1.0
7793,NaN,NaN,15499,semi-auto,64315.0,diesel,145.0,54.3,2.0,3.0
12893,NaN,NaN,14999,manual,29053.0,diesel,20.0,64.2,2.1,5.0
13087,NaN,NaN,16500,manual,3000.0,diesel,145.0,57.7,1.6,1.0


In [8]:
df_train = df_train[df_train["brand"].notnull()]
df_train.shape

(75930, 10)

In [9]:
df_train[df_train["model"].isnull()]

,brand,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age
64,ford,NaN,9999,manual,10000.0,petrol,145.0,47.9,NaN,1.0
89,ford,NaN,7099,manual,74176.0,diesel,NaN,NaN,1.5,5.0
131,opel,NaN,8695,manual,39460.0,petrol,125.0,50.4,1.4,4.0
153,ford,NaN,9632,manual,18814.0,petrol,20.0,62.8,1.0,4.0
186,opel,NaN,6999,manual,45367.0,petrol,30.0,55.4,1.4,4.0
...,...,...,...,...,...,...,...,...,...,...
75751,ford,NaN,19289,manual,28492.0,diesel,145.0,56.5,2.0,3.0
75788,opel,NaN,15495,manual,3129.0,petrol,145.0,39.2,1.4,1.0
75824,opel,NaN,14295,manual,14609.0,petrol,145.0,39.2,1.4,NaN
75898,audi,NaN,27790,automatic,6000.0,diesel,145.0,46.3,2.0,NaN


**Plan**

1. **Model** - Moda por agrupamento em Brand, ou fazer knnimputer

# Model

### Data Scaling | Dealing with Categorical Features | Outliers

### GUILERMI TRANSFORMERS

In [19]:
class RareLabelGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, min_prop=0.005, column=None):
        self.min_prop = min_prop
        self.column = column
        self.keep_values_ = None
    def fit(self, X, y=None):
        s = X if isinstance(X, pd.Series) else pd.Series(X.iloc[:,0] if hasattr(X, "iloc") else X[:,0])
        vc = s.value_counts(dropna=False)
        total = len(s)
        keep = vc[vc / total >= self.min_prop].index
        self.keep_values_ = set(keep.tolist())
        return self
    def transform(self, X):
        s = X if isinstance(X, pd.Series) else pd.Series(X.iloc[:,0] if hasattr(X, "iloc") else X[:,0])
        return pd.DataFrame(np.where(s.isin(self.keep_values_), s, "Other"),
                            columns=[self.column if self.column else "col"])

class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column): self.column = column
    def fit(self, X, y=None): return self
    def transform(self, X): return pd.DataFrame(X[self.column])

class CustomWinsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, variables, lower_bound=0.01, upper_bound=0.99):
        self.variables = variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.limits_ = {}

    def fit(self, X, y=None):
        for var in self.variables:
            if var in X.columns:
                lower_limit = X[var].quantile(self.lower_bound)
                upper_limit = X[var].quantile(self.upper_bound)
                self.limits_[var] = (lower_limit, upper_limit)
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for var, (lower_limit, upper_limit) in self.limits_.items():
            if var in X.columns:
                X_transformed[var] = np.clip(X_transformed[var], lower_limit, upper_limit)
        return X_transformed

### PIPELINE LEARNING AND CUSTOM TRANSFORMERS

In [10]:
# removing reformed cars
df_train = df_train[~(df_train["car_age"] > 45)]
df_train[df_train["car_age"] > 45]

,brand,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age


In [11]:
feature_cols = [c for c in df_train.columns if c != "price"]

X = df_train[feature_cols].copy()
y = df_train["price"].copy()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=44, shuffle=True
)

Log the target variable since it´s a variable whit a great tail (a lot of outliers)

In [12]:
y_train_transformed = np.log1p(y_train)
y_val_transformed = np.log1p(y_val)

In [13]:
# BaseEstimator → dá compatibilidade com pipelines, grid-search e clonagem
# TransformerMixin → fornece fit_transform() automaticamente
class Mode_Imput_By_Brand(BaseEstimator, TransformerMixin):

    """
    O objetivo com este transformer é preencher os valores nulos de cada coluna através da moda, 
    mas primeiro, agrupar por brand.
    Como fallback, caso nao haja nenhuma linha preenchida daquela brand em especifico, depois de utilizar este transformer
    iremos utilzar o SimpleImputer.
    
    """
    
    def __init__(self, imput_cols = None , group_col = "brand"):       
        self.imput_cols = imput_cols
        self.group_col = group_col

    # é obrigatorio colocar X e y em todos os metodos de fit do sklearn
    def fit(self , X, y = None):
        X = X.copy()  # em qualquer metodo do sklearn criamos uma copia do df original, pois no pipeline é usado o mesmo df nao modificado para os vários passos
        
        if self.imput_cols is None: # sklearn nao gosta de args obrigatórios entao fazemos isto para contornar
            self.imput_cols = [col for col in X.columns if col != self.group_col]
        
        self.mode_maps_ = {}

        for col in self.imput_cols:
            mode_map = X.groupby(self.group_col)[col].agg(lambda x: x.mode().iloc[0]) 
            self.mode_maps_[col] = mode_map 

        return self # o .fit do sklearn retorna sempre o self
    
    def transform(self, X):
        X = X.copy()

        for col in self.imput_cols:
            X[col] = X[col].fillna(X[self.group_col].map(self.mode_maps_[col]))

        return X

In [14]:
test = X_train.copy()
test["model"].isnull().sum()

1334

In [18]:
df = pd.DataFrame({
    "brand": ["A", "A", "B", "B", "C"],
    "model": ["x1", None, "y1", None, "z1"]
})

imp = Model_Mode_Imput(imput_cols=["model"], group_col="brand")
imp.fit(df)
print(imp.mode_maps_)      

print(imp.transform(df)) 

type(imp)

{'model': brand
A    x1
B    y1
C    z1
Name: model, dtype: object}
  brand model
0     A    x1
1     A    x1
2     B    y1
3     B    y1
4     C    z1


__main__.Model_Mode_Imput

## Rest

Scaling

- **Why scale:** Linear/regularized models are scale-sensitive. Numeric features can have very different ranges (ex: mileage vs engine size), which biases coefficients.

- **Choice:** **StandardScaler** for numeric columns (after imputation and outlier capping).

- **StandardScaler over MinMaxScaler:** Standardization centers and normalizes variance—well-aligned with Ridge/Lasso penalties and keeps One-Hot Enconding interpretable. All scaling is fit on the training fold only within the pipeline to ensure that there are **no leakage**.

In [20]:
scaler = StandardScaler()

In [21]:
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [c for c in X_train.columns if c not in numeric_cols]
without_model = [c for c in X_train.columns if c not in numeric_cols + ['model']]
skewness = X_train[numeric_cols].skew(numeric_only=True).sort_values(ascending=False)
log_candidates = [c for c, s in skewness.items() if np.isfinite(s) and s > 1.0]
log_candidates = [c for c in log_candidates if (X_train[c].dropna() >= 0).all()]
outlier_variable = ['tax']

In [22]:
def log1p_selective(df_num):
    df_num = pd.DataFrame(df_num, columns=numeric_cols)
    for c in log_candidates:
        df_num[c] = np.log1p(df_num[c])
    return df_num

log_transformer = FunctionTransformer(log1p_selective, validate=False)

numeric_pipeline = Pipeline([('winsor', CustomWinsorizer(variables=outlier_variable, lower_bound=0.01, upper_bound=0.99)), ("imputer", SimpleImputer(strategy="median")),("log1p_skewed", log_transformer), ("scaler", scaler)])

In [23]:
cats = []
cats.append(("cat",
                 Pipeline([("impute", SimpleImputer(strategy="most_frequent")),
                           ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False))]),
                without_model))
cats.append(("model_collap",
                 Pipeline([("select", ColumnSelector("model")),
                           ("impute", SimpleImputer(strategy="most_frequent")),
                           ("rare", RareLabelGrouper(min_prop=0.005, column="model")),
                           ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False))]),
                 ["model"]))

In [24]:
preprocessor = ColumnTransformer([("num", numeric_pipeline, numeric_cols), *cats], remainder="drop")

In [25]:
tmp = Pipeline([("prep", preprocessor)]).fit(X_train, y_train)
n_total = tmp.transform(X_train.iloc[:5]).shape[1]

In [26]:
n_total

66

## 5. Feature Selection

Following initial data preprocessing steps, which included the **removal of several irrelevant columns**, we employed **Recursive Feature Elimination (RFE)** to determine the optimal feature subset for our linear regression model. 

A **holdout** validation approach was utilized to evaluate model performance with varying numbers of features, using the **Mean Absolute Error (MAE)** as the primary evaluation metric. 
- This process identified an optimal subset consisting of 66 features.

In [27]:
grid = np.arange(1,n_total+1)

In [28]:
records = []
best_model = None
high_score=None
n_feat=0

In [29]:
for n_sel in grid:
    model = LinearRegression()
    pipe = Pipeline([
        ("impute_categoricals_grouped_by_brand", Mode_Imput_By_Brand(imput_cols = categorical_cols)),
        ("prep", preprocessor),
        ("rfe", RFE(estimator=model, n_features_to_select=n_sel)),
        ("linreg", LinearRegression()),
    ])
    pipe.fit(X_train, y_train_transformed)
    pred_log = pipe.predict(X_val)
    pred_original_scale = np.expm1(pred_log)
    mae = mean_absolute_error(y_val, pred_original_scale)
    records.append({"n_selected": n_sel, "MAE": mae})
    score = mae
    if (high_score is None) or (score < high_score):
        high_score = score
        best_model = pipe
df_best = pd.DataFrame(records).sort_values(["MAE"]).reset_index(drop=True)
print("Optimum number of features: %d" % df_best.iloc[0]['n_selected'])
print("Score with %d features: %f" % (df_best.iloc[0]['n_selected'], df_best.iloc[0]['MAE']))

KeyboardInterrupt: 

## 6. Model and evaluation

The project focuses on a regression problem with the objective of predicting used car prices. The following ML algorithms were selected for evaluation: 
- **Linear Regression:** Used as a baseline model due to its simplicity and interpretability.
- **Lasso Regression:** Explored for its ability to perform automatic feature selection by driving some coefficients to zero.
- **Decision Tree Regressor:** Implemented to capture potential non-linear relationships and more complex interactions between features, ultimately yielding the best performance.

**Assessment Strategy:** A consistent holdout split with a fixed random_state was maintained throughout the work to ensure reproducibility and avoid data leakage.

We start by implementing the optimal model (for linear regression) identified in the previous section.

In [ ]:
pred_log = best_model.predict(X_val) 
pred_original_scale = np.expm1(pred_log)

#### Metrics

- Primary metric: **MAE**, because it is in price units, offering direct interpretability, and is less sensitive to outliers compared with other metrics (ex:MSE).

- Secondary: **RMSE** (penalizes large errors) and **R²** (variance explained, for context).

In [ ]:
mae = mean_absolute_error(y_val, pred_original_scale)
rmse = mean_squared_error(y_val, pred_original_scale, squared=False)
r2 = r2_score(y_val, pred_original_scale)
print("\n--- Model Performance (Best RFE) ---")
print(f"Main Metric (MAE): {high_score:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")

#### Other models

In [ ]:
models_to_test = {
    "Lasso": LassoCV(cv=5, random_state=44),
    "DecisionTree": DecisionTreeRegressor(random_state=44)
}

high_score_2 = None
best_model_2 = None
best_model_2_name = ""
records = []

for name, model in models_to_test.items():
    pipe = Pipeline([
        ("prep", preprocessor),
        ("model", model),
    ])

    pipe.fit(X_train, y_train_transformed)
    pred_log = pipe.predict(X_val)    
    pred_original_scale = np.expm1(pred_log)

    #metrics
    mae = mean_absolute_error(y_val, pred_original_scale)
    rmse = mean_squared_error(y_val, pred_original_scale, squared=False)
    r2 = r2_score(y_val, pred_original_scale)

    records.append({"Model": name, "MAE": mae, "RMSE": rmse, "R2": r2})

    if (high_score_2 is None) or (mae < high_score_2):
        high_score_2 = mae
        best_model_2 = pipe
        best_model_2_name = name

print("Results for all models:")
results_df = pd.DataFrame(records).sort_values(by="MAE").reset_index(drop=True)
print(results_df)

print(f"Best model: {best_model_2_name}")
print(f"MAE metric for the best model: {high_score_2:.4f}")

In [ ]:
pred_log = best_model_2.predict(X_val) 
pred_original_scale = np.expm1(pred_log)

#### Plot some visualizations

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_val, y=pred_original_scale, alpha=0.6)
lims = [min(y_val.min(), pred_original_scale.min()), max(y_val.max(), pred_original_scale.max())]
plt.plot(lims, lims, color='red', linestyle='--', linewidth=2)

plt.title('Actual vs Predicted Car price')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.show()

In [ ]:
residuals = y_val - pred_original_scale

plt.figure(figsize=(8, 6))
sns.scatterplot(x=pred_original_scale, y=residuals, alpha=0.6)

plt.axhline(y=0, color='red', linestyle='--', linewidth=2)

plt.title('Residuals vs Predicted Car Prices')
plt.xlabel('Predicted Price')
plt.ylabel('Residuals')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(residuals, kde=True, bins=50)
plt.title('Distribution of Residuals')
plt.xlabel('Residual Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#Export the best model for use in test.csv dataset
#joblib.dump(best_model_2, 'best_model')